### Libraries and INitializing PySpark

In [1]:
from pyspark import SparkContext

# sc is short for spark context
sc = SparkContext(appName="IntroToSpark")

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
taxi_entry = sc.parallelize(['2009-01-01', 0, 0, 24])

In [ ]:
print(taxi_entry)

In [ ]:
ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [ ]:
from pyspark import SparkContext

sc = SparkContext(appName="IntroTo Spark")# < write code here >
weather_entry = sc.parallelize(['2009-01-01', 15.1, 26.1])# < write code here >
print(weather_entry.take(3))

In [ ]:
from pyspark.sql import SparkSession

APP_NAME = 'sampleApp'

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

### DataFrames in PySpark

In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

APP_NAME = "DataFrames"
SPARK_URL = "local[*]"

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

taxi_df = pd.read_csv('/datasets/pickups_terminal_5.csv')
taxi = spark.createDataFrame(taxi_df)

print(taxi)

In [ ]:
[Row(date='2009-01-01', hour=0, minute=0, pickups=24.0), 
Row(date='2009-01-01', hour=0, minute=30, pickups=35.0), 
Row(date='2009-01-01', hour=1, minute=0, pickups=25.0), 
Row(date='2009-01-01', hour=1, minute=30, pickups=25.0), 
Row(date='2009-01-01', hour=2, minute=0, pickups=16.0)]

In [ ]:
# you can break the line using the symbol '\'
spark = SparkSession.builder.appName(APP_NAME) \
        .config('spark.ui.showConsoleProgress', 'false') \
        .getOrCreate()

In [ ]:
# format='csv' - specify file format
# header='true' - specify that there is a header (column names) in the file
# inferSchema='true' - specify that data types should be inferred
taxi = spark.read.load('/datasets/pickups_terminal_5.csv', 
                       format='csv', header='true', inferSchema='true')

In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

APP_NAME = "DataFrames"
SPARK_URL = "local[*]"

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

taxi = spark.read.load('/datasets/pickups_terminal_5.csv', format='csv', header='true', inferSchema='true')# < write code here >

print(taxi.show(5))# < write code here >

In [ ]:
print(taxi.count()) #total number of rows in the dataframe

In [ ]:
print(taxi[['date', 'hour', 'minute']].show(5)) #susbset columns

### Missing Data in Spark

In [ ]:
print(taxi.describe().show())

In [ ]:
taxi = taxi.dropna()
print(taxi.count())# < write code here >

In [ ]:
taxi = taxi.fillna(0)
print(taxi.describe().show())

### EDA in PySpark SQL

In [ ]:
print(taxi.summary().show())

*For future reference*
We observe that the pickups data is skewed to the right, since the mean is greater than the median (the row with "50%"). We also notice that the maximum value is 310, the mean is 29, and the standard deviation is 22.45. So the maximum is significantly more than three standard deviations from the mean. Therefore, it is clearly an outlier. 

In [ ]:
taxi.registerTempTable("taxi")

In [ ]:
print(spark.sql("SELECT COUNT(*) FROM taxi WHERE pickups > 100").show())

+--------+
|count(1)|
+--------+
|    1431|
+--------+

In [ ]:
print(spark.sql("SELECT MIN(CAST(date as DATE)), MAX(CAST(date as DATE)) FROM taxi").show())

+-----------------------+-----------------------+
|min(CAST(date AS DATE))|max(CAST(date AS DATE))|
+-----------------------+-----------------------+
|             2009-01-01|             2016-06-30|
+-----------------------+-----------------------+

#Over seven years!!

In [ ]:
print(spark.sql("SELECT * FROM taxi ORDER BY pickups DESC").show(5))

+-------------------+----+------+-------+
|               date|hour|minute|pickups|
+-------------------+----+------+-------+
|2015-11-01 00:00:00|   1|    30|  310.0|
|2010-09-23 00:00:00|  22|    30|  288.0|
|2012-03-07 00:00:00|  21|     0|  268.0|
|2011-03-02 00:00:00|  20|    30|  264.0|
|2011-03-02 00:00:00|  18|    30|  263.0|
+-------------------+----+------+-------+
only showing top 5 rows

In [ ]:
print(spark.sql("SELECT COUNT(DISTINCT date) FROM taxi WHERE pickups > 200").show())# < write your code here >

+--------------------+
|count(DISTINCT date)|
+--------------------+
|                  21|
+--------------------+